In [1]:
# Import necessary packages
import pandas as pd
from pymongo import MongoClient
import time
import psutil
import cudf
from extract_load_functions_mongodb import extraction, loading

#set export location
exportLocation = r'/home/jeff/'

#set number of test runs to prefrom
iterations = 31

# Connect to MongoDb
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ds7330 
item = db.item
order_line = db.order_line

etlTimerStart = time.perf_counter()    
# MEDIUM EXTRACTION #
###################################################################################################################
### Pull item table and capture CPU, RAM and elapsed time to perform operations

extractionTimerStart = time.perf_counter()
### Run 30 iterations to collect a df of performance metrics
### light extraction performance metrics
column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
medExtractionPrfm = pd.DataFrame(columns = column_names)

### base metrics
print("STARTING EXTRACTION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

### Run n iterations to collect a df of performance metrics
for sampleNo in range(iterations):

    #Initiate timer for query
    start = time.perf_counter()

    #insert data into dataFrame
    itemdf = pd.DataFrame(list(item.find()))
    order_linedf = pd.DataFrame(list(order_line.find()))
    
    #Stop timer  
    stop = time.perf_counter()

    #load df with performance metrics    
    medExtractionPrfm = medExtractionPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                               'RAM_utilization':  psutil.virtual_memory().percent,
                                                               'elapsed_time': stop - start},
                                                              index=[1]), ignore_index=True)
    time.sleep(2)
extractionTimerEnd= time.perf_counter()   

print("Data frame loading complete.\n")
print("Total EXTRACTION time: ", extractionTimerEnd - extractionTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average EXTRACTION iteration time: ", medExtractionPrfm.elapsed_time.mean(), "s")
print("Average CPU utilization: ", medExtractionPrfm.CPU_utilization.mean())
print("Average RAM utilization: ", medExtractionPrfm.RAM_utilization.mean(),"\n\n")

#Export DF to csv
medExtractionPrfm.to_csv (r'/home/jeff/medExtractionPrfm_MongoDB.csv', index = False, header=True)

#### MEDIUM TRANSFORMATION ####
###################################################################################################################
# Preform medium transformation workload.  In this case, join orderline and item tables, identify all item descriptions with "Blue" in them 
# and change them to "Navy"

#Cast datatypes to objects
order_linedf[['Line', 'eaches_qty']] = order_linedf[['Line', 'eaches_qty','Ponum']].apply(pd.to_numeric) 
order_linedf[['Order', 'orderedItem','Ponum']] = order_linedf[['Order', 'orderedItem','Ponum']].astype(str) 

itemdf[['selling_price']] = itemdf[['selling_price']].apply(pd.to_numeric)
itemdf[['orderedItem']] = itemdf[['orderedItem']].astype(str)

column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
medTransPrfm = pd.DataFrame(columns = column_names)
#lightTrans.item_desc.str.contains("^Blue")

print("STARTING TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

transTimerStart = time.perf_counter()
#Run 30 iterations to collect transformation df of performance metrics

for sampleNoTransform in range(iterations):

    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    itemtrans = itemdf.drop(index=itemdf.index[[-1]])
    order_linetrans = order_linedf.drop(index=order_linedf.index[[-1]])
    
    itemtrans =itemdf.drop(itemdf.columns[[0]], axis = 1) 
    order_linetrans = order_linedf.drop(order_linedf.columns[[0]], axis = 1) 
    
    medium = order_linetrans.merge(right=itemtrans, on="orderedItem")
    
    medium.item_desc = medium.item_desc.str.replace('Blue', 'Navy', regex=True)
    medium['extended_price'] = medium.selling_price * medium.eaches_qty
    
    #Stop timer  
    stop = time.perf_counter()
    
    
    medTransPrfm = medTransPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
    #lightTrans = orderItemJoin
    time.sleep(2)
transTimerEnd= time.perf_counter() 

print("Pandas transformation metrics captured.\n")
print("Total extraction time: ", transTimerEnd - transTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average extraction iteration time: ", medTransPrfm.elapsed_time.mean(), "s")
print("Average CPU utilization: ", medTransPrfm.CPU_utilization.mean())
print("Average RAM utilization: ", medTransPrfm.RAM_utilization.mean(), "\n\n")

# Export Light Transformation Performance to local
medTransPrfm.to_csv (r'/home/jeff/medTransPrfm_mongoDB.csv', index = False, header=True)


#### cuDF TRANSFORMATION ####
###################################################################################################################

column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
medTransPrfmCU = pd.DataFrame(columns = column_names)

print("Starting cuDF TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

#Run 30 iterations to collect transformation df of performance metrics
transCuTimerStart = time.perf_counter()
for sampleNoTransform in range(iterations):
    ## Transform pandas df to cuDF
    itemCU = cudf.DataFrame.from_pandas(itemtrans)
    order_lineCU = cudf.DataFrame.from_pandas(order_linetrans)
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    joined_dataCU = order_lineCU.merge(right=itemCU, on="orderedItem")
    
    joined_dataCU.item_desc = joined_dataCU.item_desc.str.replace('Blue', 'Navy', regex=True)
    joined_dataCU['extended_price'] = joined_dataCU.selling_price * joined_dataCU.eaches_qty
    
    #Stop timer  
    stop = time.perf_counter()
    
    medTransPrfmCU = medTransPrfmCU.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
    #lightTrans = orderItemJoin
    time.sleep(2)
transCuTimerEnd = time.perf_counter() 

print("cuDF performance metrics captured loading complete.\n")
print("Total cuDF TRANSFORMATION time: ", transCuTimerEnd - transCuTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average cuDF TRANSFORMATION iteration time: ", medTransPrfmCU.elapsed_time.mean(), "s")
print("Average cuDF TRANSFORMATION CPU utilization: ", medTransPrfmCU.CPU_utilization.mean())
print("Average cuDF TRANSFORMATION RAM utilization: ", medTransPrfmCU.RAM_utilization.mean(),"\n\n")

# Export medium cuDF Transformation Performance to local
medTransPrfmCU.to_csv (r'/home/jeff/mediumTransPrfmCU_mongoDB.csv', index = False, header=True)

#### Load data ####
###################################################################################################################
loading(myclient,'medium', medium, iterations, exportLocation, "mediumMongoDBLoad.csv")

etlTimerEnd = time.perf_counter() 
print('ETL is complete')
print('Elapsed ETL time is: ', (etlTimerEnd-etlTimerStart)/60, ' minutes')

STARTING EXTRACTION...
Base CPU utilization:  5.9
Base RAM utilization:  17.7
RUNNING...
Data frame loading complete.

Total EXTRACTION time:  108.42602702899603 s
Iterations performed:  31
Average EXTRACTION iteration time:  1.4948253174840036 s
Average CPU utilization:  3.712903225806452
Average RAM utilization:  18.738709677419354 


STARTING TRANSFORMATION...
Base CPU utilization:  2.6
Base RAM utilization:  18.7
RUNNING...
Pandas transformation metrics captured.

Total extraction time:  67.98063021899725 s
Iterations performed:  31
Average extraction iteration time:  0.19029345642000758 s
Average CPU utilization:  1.8580645161290328
Average RAM utilization:  18.932258064516127 


Starting cuDF TRANSFORMATION...
Base CPU utilization:  1.9
Base RAM utilization:  19.0
RUNNING...
cuDF performance metrics captured loading complete.

Total cuDF TRANSFORMATION time:  65.01334980899992 s
Iterations performed:  31
Average cuDF TRANSFORMATION iteration time:  0.021172020516279453 s
Average 

In [3]:
medTransPrfmCU.to_csv (r'/home/jeff/mediumTransPrfmCU_mongoDB.csv', index = False, header=True)